# Module 6 — Object Detection with YOLOv8 (Expanded)

This notebook provides a classroom-friendly end-to-end demo using Ultralytics YOLOv8. It creates a tiny synthetic dataset (images + YOLO-format annotations), installs ultralytics, shows how to create `data.yaml`, trains a YOLOv8 model for a few epochs, and runs inference. This is intended for demos — replace the synthetic dataset with Roboflow exports or real annotated images for real projects.

## 1 — Setup (install ultralytics and imports)

In [ ]:
# Install Ultralytics YOLOv8
!pip -q install ultralytics --quiet
from ultralytics import YOLO
import os
print('Ultralytics installed')
print('YOLO class available:', 'YOLO' in globals())


## 2 — Create a tiny synthetic object-detection dataset (YOLO format)

In [ ]:
from PIL import Image, ImageDraw
import random, os

root = '/mnt/data/yolo_tiny'
train_dir = os.path.join(root, 'images', 'train')
val_dir = os.path.join(root, 'images', 'val')
labels_train = os.path.join(root, 'labels', 'train')
labels_val = os.path.join(root, 'labels', 'val')
for d in [train_dir, val_dir, labels_train, labels_val]:
    os.makedirs(d, exist_ok=True)

# Create simple images with one rectangle object per image
def create_image(path_img, path_label, img_size=(416,416)):
    w,h = img_size
    img = Image.new('RGB', img_size, (255,255,255))
    draw = ImageDraw.Draw(img)
    # random rectangle
    x1 = random.randint(20, w//2)
    y1 = random.randint(20, h//2)
    x2 = random.randint(w//2, w-20)
    y2 = random.randint(h//2, h-20)
    draw.rectangle([x1,y1,x2,y2], outline=(255,0,0), width=4)
    img.save(path_img)
    # YOLO format: class x_center y_center width height (normalized)
    xc = (x1 + x2)/2.0 / w
    yc = (y1 + y2)/2.0 / h
    nw = (x2 - x1) / w
    nh = (y2 - y1) / h
    with open(path_label, 'w') as f:
        f.write(f"0 {xc:.6f} {yc:.6f} {nw:.6f} {nh:.6f}\n")

# create 12 train images and 4 val images
for i in range(12):
    create_image(os.path.join(train_dir, f'train_{i}.jpg'), os.path.join(labels_train, f'train_{i}.txt'))
for i in range(4):
    create_image(os.path.join(val_dir, f'val_{i}.jpg'), os.path.join(labels_val, f'val_{i}.txt'))

print('Created tiny YOLO dataset at', root)
print('Train images:', len(os.listdir(train_dir)), 'Val images:', len(os.listdir(val_dir)))


## 3 — Create `data.yaml` for Ultralytics training

In [ ]:
data_yaml = f"""
train: {os.path.join(root, 'images', 'train')}
val: {os.path.join(root, 'images', 'val')}

nc: 1
names: ['object']
"""
with open('/mnt/data/yolo_tiny/data.yaml','w') as f:
    f.write(data_yaml)
print('Wrote /mnt/data/yolo_tiny/data.yaml')
print(open('/mnt/data/yolo_tiny/data.yaml').read())


## 4 — Train YOLOv8 (small demo, few epochs)

In [ ]:
# Train a tiny model (yolov8n) for a few epochs — this will download yolov8n.pt if needed
model = YOLO('yolov8n.pt')
# training; set epochs small for demo
results = model.train(data='/mnt/data/yolo_tiny/data.yaml', epochs=8, imgsz=416, batch=4)
print('Training completed — results object:', type(results))


## 5 — Run inference on validation images and show predictions

In [ ]:
# Run inference on one validation image
val_img = os.path.join(root, 'images', 'val', os.listdir(os.path.join(root,'images','val'))[0])
print('Example val image:', val_img)

res = model.predict(val_img)
# show boxes with ultralytics convenience method (saves a copy with boxes)
res_plotted = res[0].plot()
from matplotlib import pyplot as plt
plt.figure(figsize=(6,6))
plt.imshow(res_plotted)
plt.axis('off')
plt.title('YOLOv8 prediction (plotted)')
plt.show()

# Print prediction details
print('Prediction boxes:', res[0].boxes.xyxy if hasattr(res[0].boxes, 'xyxy') else 'none')
print('Prediction scores:', res[0].boxes.conf if hasattr(res[0].boxes, 'conf') else 'none')


## 6 — Export and save trained model weights

In [ ]:
# The ultralytics training saves weights in ./runs/detect/train (default). Let's find the best.pt
import glob
ckpts = glob.glob('runs/detect/train*/weights/best.pt')
if len(ckpts)>0:
    ckpt = ckpts[-1]
    os.makedirs('/mnt/data/yolov8_checkpoints', exist_ok=True)
    !cp {ckpt} /mnt/data/yolov8_checkpoints/ || true
    print('Copied checkpoint to /mnt/data/yolov8_checkpoints')
else:
    print('No checkpoint found — check training output')


## 7 — Roboflow & real dataset guidance
- To use Roboflow: export your project in YOLOv8 (YOLO Darknet/YOLOv5/YOLOv8) format and download the ZIP. Upload to Colab and extract into `/content/roboflow_dataset/`.
- Update `data.yaml` paths to point to the images/train and images/val folders provided by Roboflow.
- You can also use Roboflow train endpoint or the Roboflow Python client for direct uploads.

**In-class exercise:** annotate 20–100 images in VIA or Roboflow, export YOLO format, upload, and re-run training cells with the new dataset.